In [12]:
## importing modules and loading the data
import numpy as np
import os
import nibabel as nib
import matplotlib as mlp
import MainFunctions as MF
# downloading the data
path0=os.getcwd()



In [24]:
# download the data if it doesnt exist. 
path=path0+'/content/bold5000/sub-CSI3_anat/sub-CSI3_ses-16_T1w.nii.gz'
downloaded=os.path.isfile(path)
if not downloaded:
    !pip install awscli
    os.chdir(path0)
    os.mkdir('./content')
    os.chdir('/content')
    os.mkdir('./bold5000')
    os.chdir('/content/bold5000')
    !aws s3 sync --no-sign-request s3://openneuro.org/ds001499/sub-CSI3/ses-16/anat/ /sub-CSI3_anat/
# load the data
data,hdr= MF.load_data()

True


/Users/alizare/Desktop/DIP/projectcontent/bold5000/sub-CSI3_anat/sub-CSI3_ses-16_T1w.nii.gz
